In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
from time import sleep
from random import sample

### Brief intro
The web I want to webscrap is Vinissimus.com, it is an Spanish online specialized store of wines and liquors containing information about 7093 Spanish wines, 6620 Italian wines and 2495 French wines. <br>
I will focus my efforts in first scraping the info about the Spanish wines. <br>
Due to the structure of the webpage, first I need to obtain the links of each wine, and then from each link obtain all the information I need.

### Getting the links for each wine (first page)

In [19]:
url = "https://www.vinissimus.com/en/region/spain/?content=%5B%22region%22%2C%22espana%22%5D" # english
response = requests.get(url)
response.status_code   

200

In [20]:
soup = BeautifulSoup(response.content, "html.parser")

In [21]:
# Obtaining the links

links_page_one = []
for a in soup.find_all("div", attrs={"class": "details"}):
    links_page_one.append(a.find("a").get("href"))
    
links_page_one

['/en/wine/ultreia-saint-jacques/',
 '/en/wine/blanc-pescador/',
 '/en/wine/bach-extrisimo-blanco-semi-dulce/',
 '/en/wine/jose-pariente-verdejo/',
 '/en/wine/coto-de-imaz-reserva/',
 '/en/wine/rene-barbier-vina-augusta-semi-dulce/',
 '/en/cava/rigol-brut-nature/',
 '/en/cava/codorniu-clasico-semi-sec/',
 '/en/wine/el-coto-crianza/',
 '/en/wine/vina-tondonia-reserva/',
 '/en/wine/vina-ardanza-reserva/',
 '/en/wine/pago-de-carraovejas/',
 '/en/wine/enate-rosado/',
 '/en/wine/k-naia/',
 '/en/wine/la-montesa/',
 '/en/cava/freixenet-carta-nevada-semisec/',
 '/en/wine/domaines-lupier-la-dama/',
 '/en/wine/cora-de-loxarel/',
 '/en/cava/vall-dolina-reserva-brut-nature/',
 '/en/wine/la-rioja-alta-gran-reserva-904/',
 '/en/wine/cresta-rosa/',
 '/en/wine/enate-chardonnay-234/',
 '/en/wine/gran-cerdo/',
 '/en/wine/domaines-lupier-el-terroir/',
 '/en/wine/sericis-monastrell/',
 '/en/wine/lirondo/',
 '/en/wine/cune-verdejo/',
 '/en/wine/manzanilla-la-gitana/',
 '/en/wine/psi/',
 '/en/wine/marieta/'

In [13]:
# Add the beginning of the link to all links

missing_part = "https://www.vinissimus.com"

links_page_one_compl = [missing_part + l for l in links_page_one]
links_page_one_compl

['https://www.vinissimus.com/en/wine/ultreia-saint-jacques/',
 'https://www.vinissimus.com/en/wine/blanc-pescador/',
 'https://www.vinissimus.com/en/wine/bach-extrisimo-blanco-semi-dulce/',
 'https://www.vinissimus.com/en/wine/jose-pariente-verdejo/',
 'https://www.vinissimus.com/en/wine/coto-de-imaz-reserva/',
 'https://www.vinissimus.com/en/wine/rene-barbier-vina-augusta-semi-dulce/',
 'https://www.vinissimus.com/en/cava/rigol-brut-nature/',
 'https://www.vinissimus.com/en/cava/codorniu-clasico-semi-sec/',
 'https://www.vinissimus.com/en/wine/el-coto-crianza/',
 'https://www.vinissimus.com/en/wine/vina-tondonia-reserva/',
 'https://www.vinissimus.com/en/wine/vina-ardanza-reserva/',
 'https://www.vinissimus.com/en/wine/pago-de-carraovejas/',
 'https://www.vinissimus.com/en/wine/enate-rosado/',
 'https://www.vinissimus.com/en/wine/k-naia/',
 'https://www.vinissimus.com/en/wine/la-montesa/',
 'https://www.vinissimus.com/en/cava/freixenet-carta-nevada-semisec/',
 'https://www.vinissimus.

### Getting the links for each wine (rest of the pages)
There are 198 pages. These are the urls for the pages 2, 3, 4 and 198 (Spanish):
- https://www.vinissimus.com/es/region/espana/?sort_by=%5B%7B%22field%22%3A%22opciones.in_stock%22%2C%22order%22%3A%22asc%22%7D%2C%7B%22field%22%3A%22ranking%22%2C%22order%22%3A%22asc%22%7D%5D&cursor=36
- https://www.vinissimus.com/es/region/espana/?sort_by=%5B%7B%22field%22%3A%22opciones.in_stock%22%2C%22order%22%3A%22asc%22%7D%2C%7B%22field%22%3A%22ranking%22%2C%22order%22%3A%22asc%22%7D%5D&cursor=72 
- https://www.vinissimus.com/es/region/espana/?sort_by=%5B%7B%22field%22%3A%22opciones.in_stock%22%2C%22order%22%3A%22asc%22%7D%2C%7B%22field%22%3A%22ranking%22%2C%22order%22%3A%22asc%22%7D%5D&cursor=108 
- https://www.vinissimus.com/es/region/espana/?sort_by=%5B%7B%22field%22%3A%22opciones.in_stock%22%2C%22order%22%3A%22asc%22%7D%2C%7B%22field%22%3A%22ranking%22%2C%22order%22%3A%22asc%22%7D%5D&cursor=7092

English: 
- https://www.vinissimus.com/en/region/spain/?sort_by=%5B%7B%22field%22%3A%22opciones.in_stock%22%2C%22order%22%3A%22asc%22%7D%2C%7B%22field%22%3A%22ranking%22%2C%22order%22%3A%22asc%22%7D%5D&cursor=36
- https://www.vinissimus.com/en/region/spain/?sort_by=%5B%7B%22field%22%3A%22opciones.in_stock%22%2C%22order%22%3A%22asc%22%7D%2C%7B%22field%22%3A%22ranking%22%2C%22order%22%3A%22asc%22%7D%5D&cursor=72

The last figure increases from 36 to 36.

In [22]:
# Obtain the total number of wines 

number_string = soup.find_all("span", attrs={"class":"total-count"})[0].get_text()
num = ""
for c in number_string:
    if c.isdigit():
        num = num + c
total_number = int(num)
total_number

7100

In [23]:
pages = list(range (36, total_number, 36))
pages

[36,
 72,
 108,
 144,
 180,
 216,
 252,
 288,
 324,
 360,
 396,
 432,
 468,
 504,
 540,
 576,
 612,
 648,
 684,
 720,
 756,
 792,
 828,
 864,
 900,
 936,
 972,
 1008,
 1044,
 1080,
 1116,
 1152,
 1188,
 1224,
 1260,
 1296,
 1332,
 1368,
 1404,
 1440,
 1476,
 1512,
 1548,
 1584,
 1620,
 1656,
 1692,
 1728,
 1764,
 1800,
 1836,
 1872,
 1908,
 1944,
 1980,
 2016,
 2052,
 2088,
 2124,
 2160,
 2196,
 2232,
 2268,
 2304,
 2340,
 2376,
 2412,
 2448,
 2484,
 2520,
 2556,
 2592,
 2628,
 2664,
 2700,
 2736,
 2772,
 2808,
 2844,
 2880,
 2916,
 2952,
 2988,
 3024,
 3060,
 3096,
 3132,
 3168,
 3204,
 3240,
 3276,
 3312,
 3348,
 3384,
 3420,
 3456,
 3492,
 3528,
 3564,
 3600,
 3636,
 3672,
 3708,
 3744,
 3780,
 3816,
 3852,
 3888,
 3924,
 3960,
 3996,
 4032,
 4068,
 4104,
 4140,
 4176,
 4212,
 4248,
 4284,
 4320,
 4356,
 4392,
 4428,
 4464,
 4500,
 4536,
 4572,
 4608,
 4644,
 4680,
 4716,
 4752,
 4788,
 4824,
 4860,
 4896,
 4932,
 4968,
 5004,
 5040,
 5076,
 5112,
 5148,
 5184,
 5220,
 5256,
 5292,


In [25]:
links_rest_pages = []

while True:
    try:
        for page in pages:
            # request = requests.get(f"https://www.vinissimus.com/es/region/espana/?sort_by=%5B%7B%22field%22%3A%22opciones.in_stock%22%2C%22order%22%3A%22asc%22%7D%2C%7B%22field%22%3A%22ranking%22%2C%22order%22%3A%22asc%22%7D%5D&cursor={page}")
            request = requests.get(f"https://www.vinissimus.com/en/region/spain/?sort_by=%5B%7B%22field%22%3A%22opciones.in_stock%22%2C%22order%22%3A%22asc%22%7D%2C%7B%22field%22%3A%22ranking%22%2C%22order%22%3A%22asc%22%7D%5D&cursor={page}")
            soup = BeautifulSoup(request.content, 'html.parser')
            for a in soup.find_all("div", attrs={"class": "details"}):
                links_rest_pages.append(a.find("a").get("href"))
            wait_time = np.random.randint(2,5)
            sleep(wait_time)
    except Exception as e:
        continue
    break

links_rest_pages

['/en/wine/campo-viejo-blanco/',
 '/en/wine/faustino-v-blanco/',
 '/en/wine/protos-roble/',
 '/en/wine/rioja-bordon-blanco/',
 '/en/wine/marina-alta/',
 '/en/wine/baron-de-ley-reserva/',
 '/en/wine/dehesa-la-granja/',
 '/en/cava/anna-de-codorniu-brut-nature/',
 '/en/wine/casa-castillo/',
 '/en/cava/codorniu-brut-nature/',
 '/en/wine/sio/',
 '/en/wine/marques-de-caceres-rosado/',
 '/en/wine/pescador-rose/',
 '/en/wine/murua-reserva/',
 '/en/wine/baron-de-ley-blanco-semidulce/',
 '/en/wine/ribas-negre/',
 '/en/cava/perelada-blue-festival-brut-reserva/',
 '/en/wine/antonio-barbadillo-castillo-de-san-diego/',
 '/en/wine/vins-padro-negre/',
 '/en/wine/habla-del-silencio/',
 '/en/wine/txakoli-ameztoi/',
 '/en/wine/mia-freixenet-blanco/',
 '/en/wine/maior-de-mendoza-fulget/',
 '/en/wine/enate-merlot-merlot/',
 '/en/wine/marques-de-caceres-crianza/',
 '/en/wine/mas-d-en-canet-blanc/',
 '/en/wine/la-planta/',
 '/en/wine/les-cousins-l-inconscient/',
 '/en/wine/marques-de-murrieta-reserva/',
 '/e

In [ ]:
len(links_rest_pages)

In [26]:
# Add the beginning of the link to all links

missing_part = "https://www.vinissimus.com"

links_rest_pages_compl = [missing_part + l for l in links_rest_pages]
links_rest_pages_compl

['https://www.vinissimus.com/en/wine/campo-viejo-blanco/',
 'https://www.vinissimus.com/en/wine/faustino-v-blanco/',
 'https://www.vinissimus.com/en/wine/protos-roble/',
 'https://www.vinissimus.com/en/wine/rioja-bordon-blanco/',
 'https://www.vinissimus.com/en/wine/marina-alta/',
 'https://www.vinissimus.com/en/wine/baron-de-ley-reserva/',
 'https://www.vinissimus.com/en/wine/dehesa-la-granja/',
 'https://www.vinissimus.com/en/cava/anna-de-codorniu-brut-nature/',
 'https://www.vinissimus.com/en/wine/casa-castillo/',
 'https://www.vinissimus.com/en/cava/codorniu-brut-nature/',
 'https://www.vinissimus.com/en/wine/sio/',
 'https://www.vinissimus.com/en/wine/marques-de-caceres-rosado/',
 'https://www.vinissimus.com/en/wine/pescador-rose/',
 'https://www.vinissimus.com/en/wine/murua-reserva/',
 'https://www.vinissimus.com/en/wine/baron-de-ley-blanco-semidulce/',
 'https://www.vinissimus.com/en/wine/ribas-negre/',
 'https://www.vinissimus.com/en/cava/perelada-blue-festival-brut-reserva/',


In [27]:
# Join the two lists  

links = links_page_one_compl + links_rest_pages_compl
links

['https://www.vinissimus.com/en/wine/ultreia-saint-jacques/',
 'https://www.vinissimus.com/en/wine/blanc-pescador/',
 'https://www.vinissimus.com/en/wine/bach-extrisimo-blanco-semi-dulce/',
 'https://www.vinissimus.com/en/wine/jose-pariente-verdejo/',
 'https://www.vinissimus.com/en/wine/coto-de-imaz-reserva/',
 'https://www.vinissimus.com/en/wine/rene-barbier-vina-augusta-semi-dulce/',
 'https://www.vinissimus.com/en/cava/rigol-brut-nature/',
 'https://www.vinissimus.com/en/cava/codorniu-clasico-semi-sec/',
 'https://www.vinissimus.com/en/wine/el-coto-crianza/',
 'https://www.vinissimus.com/en/wine/vina-tondonia-reserva/',
 'https://www.vinissimus.com/en/wine/vina-ardanza-reserva/',
 'https://www.vinissimus.com/en/wine/pago-de-carraovejas/',
 'https://www.vinissimus.com/en/wine/enate-rosado/',
 'https://www.vinissimus.com/en/wine/k-naia/',
 'https://www.vinissimus.com/en/wine/la-montesa/',
 'https://www.vinissimus.com/en/cava/freixenet-carta-nevada-semisec/',
 'https://www.vinissimus.

In [ ]:
len(links)

In [28]:
df = pd.DataFrame(links)
df

,0
0,https://www.vinissimus.com/en/wine/ultreia-sai...
1,https://www.vinissimus.com/en/wine/blanc-pesca...
2,https://www.vinissimus.com/en/wine/bach-extris...
3,https://www.vinissimus.com/en/wine/jose-parien...
4,https://www.vinissimus.com/en/wine/coto-de-ima...
...,...
7095,https://www.vinissimus.com/en/wine/augustus-mi...
7096,https://www.vinissimus.com/en/cava/marrugat-br...
7097,https://www.vinissimus.com/en/wine/tianna-velo...
7098,https://www.vinissimus.com/en/cava/gran-campo-...


In [29]:
df.to_csv("links.csv")

In [8]:
links = pd.read_csv("links.csv")
links = list(links["0"])
links

['https://www.vinissimus.com/en/wine/ultreia-saint-jacques/',
 'https://www.vinissimus.com/en/wine/blanc-pescador/',
 'https://www.vinissimus.com/en/wine/terras-gauda/',
 'https://www.vinissimus.com/en/wine/coto-de-imaz-reserva/',
 'https://www.vinissimus.com/en/wine/rene-barbier-vina-augusta-semi-dulce/',
 'https://www.vinissimus.com/en/wine/el-coto-crianza/',
 'https://www.vinissimus.com/en/wine/vina-tondonia-reserva/',
 'https://www.vinissimus.com/en/wine/vina-ardanza-reserva/',
 'https://www.vinissimus.com/en/wine/aalto/',
 'https://www.vinissimus.com/en/wine/anima-negra-an-2/',
 'https://www.vinissimus.com/en/cava/rigol-brut-nature/',
 'https://www.vinissimus.com/en/wine/pago-de-carraovejas/',
 'https://www.vinissimus.com/en/cava/freixenet-carta-nevada-semisec/',
 'https://www.vinissimus.com/en/wine/ribas-rosat/',
 'https://www.vinissimus.com/en/cava/freixenet-reserva-brut-nature/',
 'https://www.vinissimus.com/en/cava/vall-dolina-reserva-brut-nature/',
 'https://www.vinissimus.co

### Get the info from every wine

In [16]:
# Functions for exception handling when there is no element to retrieve

def get_element(lst, index):
    try:
        lst[index]
    except IndexError:
        return ""
    else:
        return lst[index].get_text()
    
def get_price(lst, index):
    try:
        element = lst[index].get_text()
    except IndexError:
        element = soup.find_all("div", attrs={"class": "pprice"})[0].find("p").get_text()
    
    return element

def get_volume(soup):
    try:
        element = soup.find_all("div", attrs={"class": "quantity-widget large has-0-flags"})[0].select("span", attrs={"class": "unit-name  large"})[2].get_text()
    except IndexError:
        element = soup.find_all("div", attrs={"class": "pprice"})[0].find_all("span", attrs={"class":"unit-name red large"})[0].get_text()
    
    return element

def get_tasting(lst, index):
    try:
        element = lst[0]
    except IndexError:
        return ""
    else:
        return element.find_all("td")[index].get_text()
    
def get_percentages(lst, index):
    try: 
        element = lst[index]
    except IndexError:
        return ""
    else:
        return element.find_all("span")[0]

def get_technical_info(lst, index1, index2):
    try:
        element = lst[index1]
    except IndexError:
        return ""
    else:
        try:
            td_element = element.select("td")[index2]
        except IndexError:
            return ""
        else:
            return td_element.get_text()
    
def get_score(lst, index, score_name):
    try:
        element = lst[index]
    except IndexError:
        return ""
    else:
        try: 
            td_element = element.find("td", attrs={"title": score_name})
        except AttributeError:
            return ""
        else:
            if td_element is not None:
                return td_element.get_text()
            else:
                return ""

def get_score_year(lst, index):
    try:
        lst[index]
    except IndexError:
        return ""
    else:
        return lst[index].find("th").get_text()    

In [35]:
# Function for obtaining all the info

def obtain_wine_info(soup):
    
    # Define lists for each feature I want to obtain
    name = []
    price = []
    volume = []
    image = []
    agriculture = []
    
    wine_type = []
    region = []
    grapes = []
    producer = []
    alergens = []
    alcohol = []
    production = []
    material = []
    aging_period = []
    casks_age = []
    casks_wood = []
    winemake = []
    vineyard_age = []
    soil_type = []
    climate = []
    orientation = []
    surface = []

    spec_sheet = [wine_type, region, grapes, producer, alergens, alcohol, production]
    winemaking = [material, aging_period, casks_age, casks_wood, winemake]
    vineyards = [vineyard_age, soil_type, climate, orientation, surface]
    technical_info = [spec_sheet, winemaking, vineyards]
    
    wine_descr = []
    tasting_descr = []
    conditions_descr = []
    winemake_descr = []
    vineyards_descr = []
    broad_text = [wine_descr, tasting_descr, conditions_descr, winemake_descr, vineyards_descr]
    
    serve_temp = []
    food_pairing = []
    customer_reviews = []
    customer_reviews_number = []
    last_score_parker = []
    last_score_penin = []
    last_score_year = []
    
    view = []
    bouquet = []
    mouth = []
    tasting = [view, bouquet, mouth]
    
    light_bold = []
    smooth_tannic = []
    dry_sweet = []
    soft_acidic = []
    taste_percentages = [light_bold, smooth_tannic, dry_sweet, soft_acidic]
    
    # Assing elements to each list
    name.append(soup.find_all("div", attrs={"class": "product-title"})[0].get_text()) 
    price.append(get_price(soup.find_all("p", attrs={"class": "price uniq large"}), 0)) 
    volume.append(get_volume(soup))
    image.append(soup.find_all("div", attrs={"class": "product-image desktop"})[0].find("img").get("src")) 
    agriculture.append(get_element(soup.find_all("div", attrs={"class": "badge-with-label-text large"}), 0))
    
    for i in range(len(tasting)):
        tasting[i].append(get_tasting(soup.find_all("table", attrs={"class":"styles_table__jWPKo info-grid taste"}), i))
        
    for i in range(len(taste_percentages)):
        element = str(get_percentages(soup.find_all("div", attrs={"class":"styles_bar__Y54KE"}), i))
        if element == "":
            taste_percentages[i].append(element)
        else:
            element = int(element[55:-11])
            taste_percentages[i].append(element)
    
    for i in range(len(technical_info)):
        for j in range(len(technical_info[i])):
            technical_info[i][j].append(get_technical_info(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid technical-info"}), i, j))
    
    for i in range(len(broad_text)):
        broad_text[i].append(get_element(soup.find_all("div", attrs={"class": "sanitize-html"}), i))

    element_list = soup.find_all("div", attrs={"style": "display: flex; justify-content: center; flex-direction: column;"})
    serve_temp.append(get_element(element_list, 0))
    
    element_list = soup.find_all("div", attrs={"style": "display: flex; flex-wrap: wrap; gap: 10px; line-height: 18px; font-size: 11px; text-transform: uppercase;"})
    food_pairing.append(get_element(element_list, 0))    
    
    customer_reviews.append(soup.find_all("div", attrs={"class": "styles_firstBlock__NtfrI"})[1].get_text())
    customer_reviews_number.append(soup.find_all("div", attrs={"class": "opinion-widget rating"})[0].find("span", attrs={"class":"styles_numOpinions__t_p9L"}).get_text())
    
    element_list = soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid score-awards"})
    last_score_parker.append(get_score(element_list, 0, "Parker"))
    last_score_penin.append(get_score(element_list, 0, "Peñín"))
    last_score_year.append(get_score_year(element_list, 0))

    # Create dictionary that stores all elements
    dct = {
        "name": name,
        "price": price,
        "bottle": volume,
        "type_wine": wine_type,
        "region": region,
        "grapes": grapes,
        "producer": producer,
        "wine_description": wine_descr,
        "tasting_descr": tasting_descr,
        "tasting_view": view,
        "tasting_bouquet": bouquet, 
        "tasting_mouth": mouth, 
        "light_bold_perc": light_bold, 
        "smooth_tannic_perc": smooth_tannic,
        "dry_sweet_perc": dry_sweet,
        "soft_acidic_perc": soft_acidic,
        "serving_storage_conditions": conditions_descr,  
        "serve_temperature": serve_temp,
        "food_pairing": food_pairing,
        "alergens": alergens,
        "alcohol_content": alcohol,
        "production": production,
        "winemaking_info": winemake_descr,
        "vessel_material": material,
        "aging_period": aging_period,
        "casks_age": casks_age,
        "casks_wood": casks_wood,
        "more_winemaking_info": winemake,
        "vineyards": vineyards_descr,
        "type_agriculture": agriculture,
        "vineyard_age": vineyard_age,
        "soil_type": soil_type,
        "climate": climate,
        "orientation": orientation,
        "surface": surface,
        "customer_reviews": customer_reviews,
        "customer_reviews_number": customer_reviews_number,
        "last_score_parker": last_score_parker,
        "last_score_penin": last_score_penin,
        "last_score_year": last_score_year,
        "image": image
    }

    return dct

In [37]:
# Create loop to webscrap all pages

wine_data = pd.DataFrame()
wrong_links = []

for link in links:
    try:
        request = requests.get(link)
        soup = BeautifulSoup(request.content, 'html.parser')
        info_dct = obtain_wine_info(soup)
    except Exception as e:
        wrong_links.append(link)
        print(link)
        print(e)
    else:
        new_df = pd.DataFrame.from_dict(info_dct)
        new_df["url"] = link
        wine_data = pd.concat([wine_data, new_df])    
        wait_time = np.random.randint(3,7)
        sleep(wait_time)

wine_data

https://www.vinissimus.com/en/wine/pescador-rose/
list index out of range
https://www.vinissimus.com/en/wine/baron-de-ley-blanco-semidulce/
list index out of range
https://www.vinissimus.com/en/wine/bozeto-de-exopto/
list index out of range
https://www.vinissimus.com/en/wine/a-dos-manos/
list index out of range
https://www.vinissimus.com/en/wine/doix-murmuri/
list index out of range
https://www.vinissimus.com/en/wine/poesia-garnacha-tinta/
list index out of range
https://www.vinissimus.com/en/cava/1-1-3-ardea-rose-brut-reserva/
list index out of range
https://www.vinissimus.com/en/wine/la-forcalla-de-antonia/
list index out of range
https://www.vinissimus.com/en/wine/aseginolaza-leunda-cuvee-las-santas/
list index out of range
https://www.vinissimus.com/en/wine/kina-san-clemente-100cl/
list index out of range
https://www.vinissimus.com/en/cava/gramona-argent-gran-reserva/
list index out of range
https://www.vinissimus.com/en/wine/alto-acin-verdejo/
list index out of range
https://www.v

https://www.vinissimus.com/en/wine/albamar-o-sebal/
list index out of range
https://www.vinissimus.com/en/wine/fuentes-del-silencio-cepas-viejas/
list index out of range
https://www.vinissimus.com/en/wine/milu/
list index out of range
https://www.vinissimus.com/en/wine/emporion/
HTTPSConnectionPool(host='www.vinissimus.com', port=443): Max retries exceeded with url: /en/wine/emporion/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000228A87271F0>: Failed to establish a new connection: [WinError 10060] Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente tras un periodo de tiempo, o bien se produjo un error en la conexión establecida ya que el host conectado no ha podido responder'))
https://www.vinissimus.com/en/wine/bruma-del-estrecho-paraje-las-chozas/
list index out of range
https://www.vinissimus.com/en/wine/milu/
list index out of range
https://www.vinissimus.com/en/wine/mas-de-leda/
list index ou

,name,price,bottle,type_wine,region,grapes,producer,wine_description,tasting_descr,tasting_view,...,climate,orientation,surface,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,image,url
0,Ultreia Saint Jacques 2021,£16.20,/ 0.75 L btl,Red wine,Bierzo (Spain),100% Mencia,Raúl Pérez Viticultor,Ultreia Saint Jacques is one of the best wines...,"Bottled unclarified and unfiltered, Ultreia Sa...",Intense,...,Continental with Atlantic influence,East and south,2.00 hectares,4.2/5,217 reviews,93-94 PK,,2020,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/ultreia-sai...
0,Blanc Pescador,£7.80,/ 0.75 L btl,Frizzante white wine,Wines without GI (Spain),61% Macabeo6% Parellada33% Xarel·lo,Perelada,Blanc Pescador and Cresta Rosa have been bench...,"A pale colour, with light yellow-green hues an...",Straw yellow / Pale / Greenish hues / Bright,...,,,,4.4/5,45 reviews,,,,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...
0,Bach Extrísimo Blanco Semidulce 2021,£9.40,/ 0.75 L btl,White wine,Catalunya (Spain),Xarel·loMacabeo,Masia Bach,The Extrísimo Semidulce from Masia Bach is a h...,"In 1915, brothers Pere and Ramon Bach i Escope...",Yellow / Golden glints / Clean / Bright,...,,,,4.8/5,95 reviews,,,,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/bach-extris...
0,José Pariente Verdejo 2022,£16.20,/ 0.75 L btl,White wine,Rueda (Spain),100% Verdejo,Bodegas José Pariente,The José Pariente Verdejo is one of the great ...,A golden colour in the glass with greenish hig...,Straw yellow / Greenish glints,...,,,,4.4/5,147 reviews,,,2021,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/jose-parien...
0,Coto de Imaz Reserva 2018,£13.00,/ 0.75 L btl,Red wine,Rioja (Spain),100% Tempranillo,El Coto de Rioja,The Coto de Imaz Reserva is an intense and sil...,"At first sight, it appears a vivid cherry colo...",Picota cherry / Intense / Garnet rim,...,,,,4.4/5,94 reviews,,,2018,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/coto-de-ima...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Augustus Microvinificacions Xarel·lo de Mar 2015,£18.40,,White wine,Penedès (Spain),100% Xarel·lo,Celler Avgvstvs Forvm,Cellers Avgvstvs Forvm is a small family-owned...,,Pale yellow / Intense,...,Northwest,,,0.0/5,0 reviews,,,,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/augustus-mi...
0,Marrugat Brut Imperial,£14.45,,Cava,DO Cava (Spain),MacabeoParelladaXarel·lo,Caves Marrugat,,,Straw yellow / Clean / Bright / Fine and persi...,...,,,,0.0/5,0 reviews,,,,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/cava/marrugat-br...
0,Tianna Vélorosé 2020,£17.45,,Rosé wine,Binissalem-Mallorca (Spain),100% Mantonegro,Celler Tianna Negre,,,Salmon / Pale,...,,,,5.0/5,1 review,,,,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/tianna-velo...
0,Gran Campo Viejo Cava Brut Reserva,£12.40,,Cava,DO Cava (Spain),ParelladaMacabeoXarel·lo,Bodegas Campo Viejo,Campo Viejo is one of the Rioja region's most ...,,,...,,,,4.4/5,5 reviews,,,,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/cava/gran-campo-...


In [ ]:
# https://stackoverflow.com/questions/34315388/python-web-scraping-error-10060

In [38]:
# Save the csv

wine_data.to_csv("wine_data.csv", index=False)

In [39]:
wine_data

,name,price,bottle,type_wine,region,grapes,producer,wine_description,tasting_descr,tasting_view,...,climate,orientation,surface,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,image,url
0,Ultreia Saint Jacques 2021,£16.20,/ 0.75 L btl,Red wine,Bierzo (Spain),100% Mencia,Raúl Pérez Viticultor,Ultreia Saint Jacques is one of the best wines...,"Bottled unclarified and unfiltered, Ultreia Sa...",Intense,...,Continental with Atlantic influence,East and south,2.00 hectares,4.2/5,217 reviews,93-94 PK,,2020,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/ultreia-sai...
0,Blanc Pescador,£7.80,/ 0.75 L btl,Frizzante white wine,Wines without GI (Spain),61% Macabeo6% Parellada33% Xarel·lo,Perelada,Blanc Pescador and Cresta Rosa have been bench...,"A pale colour, with light yellow-green hues an...",Straw yellow / Pale / Greenish hues / Bright,...,,,,4.4/5,45 reviews,,,,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...
0,Bach Extrísimo Blanco Semidulce 2021,£9.40,/ 0.75 L btl,White wine,Catalunya (Spain),Xarel·loMacabeo,Masia Bach,The Extrísimo Semidulce from Masia Bach is a h...,"In 1915, brothers Pere and Ramon Bach i Escope...",Yellow / Golden glints / Clean / Bright,...,,,,4.8/5,95 reviews,,,,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/bach-extris...
0,José Pariente Verdejo 2022,£16.20,/ 0.75 L btl,White wine,Rueda (Spain),100% Verdejo,Bodegas José Pariente,The José Pariente Verdejo is one of the great ...,A golden colour in the glass with greenish hig...,Straw yellow / Greenish glints,...,,,,4.4/5,147 reviews,,,2021,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/jose-parien...
0,Coto de Imaz Reserva 2018,£13.00,/ 0.75 L btl,Red wine,Rioja (Spain),100% Tempranillo,El Coto de Rioja,The Coto de Imaz Reserva is an intense and sil...,"At first sight, it appears a vivid cherry colo...",Picota cherry / Intense / Garnet rim,...,,,,4.4/5,94 reviews,,,2018,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/coto-de-ima...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Augustus Microvinificacions Xarel·lo de Mar 2015,£18.40,,White wine,Penedès (Spain),100% Xarel·lo,Celler Avgvstvs Forvm,Cellers Avgvstvs Forvm is a small family-owned...,,Pale yellow / Intense,...,Northwest,,,0.0/5,0 reviews,,,,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/augustus-mi...
0,Marrugat Brut Imperial,£14.45,,Cava,DO Cava (Spain),MacabeoParelladaXarel·lo,Caves Marrugat,,,Straw yellow / Clean / Bright / Fine and persi...,...,,,,0.0/5,0 reviews,,,,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/cava/marrugat-br...
0,Tianna Vélorosé 2020,£17.45,,Rosé wine,Binissalem-Mallorca (Spain),100% Mantonegro,Celler Tianna Negre,,,Salmon / Pale,...,,,,5.0/5,1 review,,,,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/tianna-velo...
0,Gran Campo Viejo Cava Brut Reserva,£12.40,,Cava,DO Cava (Spain),ParelladaMacabeoXarel·lo,Bodegas Campo Viejo,Campo Viejo is one of the Rioja region's most ...,,,...,,,,4.4/5,5 reviews,,,,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/cava/gran-campo-...


In [13]:
links[0]

'https://www.vinissimus.com/en/wine/ultreia-saint-jacques/'

In [14]:
links[1]

'https://www.vinissimus.com/en/wine/blanc-pescador/'

In [40]:
len(wrong_links)

84

In [42]:
df = pd.DataFrame(wrong_links)
df.to_csv("wrong_links.csv", index=False)

### NaNs in bottle size

In [2]:
bottle_nan = pd.read_csv("bottle_nan.csv")

In [3]:
bottle_nan_links = list(bottle_nan["url"])

In [4]:
bottle = pd.DataFrame()
wrong_links = []

for link in bottle_nan_links:
    try:
        request = requests.get(link)
        soup = BeautifulSoup(request.content, 'html.parser')
        wait_time = np.random.randint(3, 7)
        sleep(wait_time)
        for i in soup.find_all("div", attrs={"class": "quantity-widget large has-0-flags"})[0].select("span", attrs={"class": "unit-name  large"}):
            element = i.get_text()
            if "btl" in element:
                dict = {"url": link, "bottle": [element]}
                new_df = pd.DataFrame.from_dict(dict)
                bottle = pd.concat([bottle, new_df])
    except Exception as e:
        wrong_links.append(link)
        print(link)
        print(e)  
        
          
bottle.to_csv("bottle.csv", index=False)

https://www.vinissimus.com/en/cava/parxet-brut-reserva/
list index out of range
https://www.vinissimus.com/en/cava/mont-marcal-reserva-brut/
list index out of range
https://www.vinissimus.com/en/cava/guspira-estones-de-mishima/
list index out of range
https://www.vinissimus.com/en/cava/misterios-escumos-rosat/
list index out of range
https://www.vinissimus.com/en/wine/casa-de-fieras-crianza/
list index out of range


In [11]:
# Since all the bottles of the wrong links are of 0.75l, add the last rows 

wrong_links_df = pd.DataFrame({"url":wrong_links})
wrong_links_df["bottle"] = 0.75
wrong_links_df

,url,bottle
0,https://www.vinissimus.com/en/cava/parxet-brut...,0.75
1,https://www.vinissimus.com/en/cava/mont-marcal...,0.75
2,https://www.vinissimus.com/en/cava/guspira-est...,0.75
3,https://www.vinissimus.com/en/cava/misterios-e...,0.75
4,https://www.vinissimus.com/en/wine/casa-de-fie...,0.75


In [13]:
bottle_complete = pd.concat([bottle, wrong_links_df])  
bottle_complete.tail(7)

,url,bottle
0,https://www.vinissimus.com/en/wine/can-bas-mon...,/ 0.75 L btl
0,https://www.vinissimus.com/en/wine/can-bas-la-...,/ 0.75 L btl
0,https://www.vinissimus.com/en/cava/parxet-brut...,0.75
1,https://www.vinissimus.com/en/cava/mont-marcal...,0.75
2,https://www.vinissimus.com/en/cava/guspira-est...,0.75
3,https://www.vinissimus.com/en/cava/misterios-e...,0.75
4,https://www.vinissimus.com/en/wine/casa-de-fie...,0.75


In [14]:
bottle_complete.to_csv("bottle_final.csv", index=False)

In [ ]:
df = pd.DataFrame(wrong_links)
df.to_csv("bottle_wrong_links.csv", index=False)

### Get price info in euros (from the pages in Spanish)

In [2]:
# First page

url = "https://www.vinissimus.com/es/region/espana/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

links_page_one = []
for a in soup.find_all("div", attrs={"class": "details"}):
    links_page_one.append(a.find("a").get("href"))
    
links_page_one

['/es/vino/pago-de-carraovejas/',
 '/es/vino/jose-pariente-verdejo/',
 '/es/vino/ultreia-saint-jacques/',
 '/es/vino/psi/',
 '/es/vino/la-montesa/',
 '/es/vino/louro/',
 '/es/vino/vina-tondonia-reserva/',
 '/es/vino/vina-ardanza-reserva/',
 '/es/vino/marques-de-murrieta-reserva/',
 '/es/vino/domaines-lupier-el-terroir/',
 '/es/vino/emilio-moro-finca-resalso/',
 '/es/vino/pago-de-los-capellanes-roble/',
 '/es/vino/nekeas-cepa-por-cepa-garnacha/',
 '/es/vino/vina-tondonia-blanco-reserva/',
 '/es/vino/les-cousins-l-inconscient/',
 '/es/vino/o-luar-do-sil/',
 '/es/vino/corimbo/',
 '/es/vino/muga-crianza/',
 '/es/vino/belondrade-y-lurton/',
 '/es/vino/pago-de-los-capellanes-crianza/',
 '/es/vino/enate-chardonnay-234/',
 '/es/vino/el-castro-de-valtuille-mencia-joven/',
 '/es/vino/hacienda-solano-seleccion/',
 '/es/vino/els-pics/',
 '/es/vino/les-terrasses-laderas-de-pizarra/',
 '/es/vino/laya/',
 '/es/vino/tinto-pesquera-crianza/',
 '/es/cava/familia-oliveda-brut-jove/',
 '/es/vino/casa-cast

In [3]:
missing_part = "https://www.vinissimus.com"

links_page_one_compl = [missing_part + l for l in links_page_one]
links_page_one_compl

['https://www.vinissimus.com/es/vino/pago-de-carraovejas/',
 'https://www.vinissimus.com/es/vino/jose-pariente-verdejo/',
 'https://www.vinissimus.com/es/vino/ultreia-saint-jacques/',
 'https://www.vinissimus.com/es/vino/psi/',
 'https://www.vinissimus.com/es/vino/la-montesa/',
 'https://www.vinissimus.com/es/vino/louro/',
 'https://www.vinissimus.com/es/vino/vina-tondonia-reserva/',
 'https://www.vinissimus.com/es/vino/vina-ardanza-reserva/',
 'https://www.vinissimus.com/es/vino/marques-de-murrieta-reserva/',
 'https://www.vinissimus.com/es/vino/domaines-lupier-el-terroir/',
 'https://www.vinissimus.com/es/vino/emilio-moro-finca-resalso/',
 'https://www.vinissimus.com/es/vino/pago-de-los-capellanes-roble/',
 'https://www.vinissimus.com/es/vino/nekeas-cepa-por-cepa-garnacha/',
 'https://www.vinissimus.com/es/vino/vina-tondonia-blanco-reserva/',
 'https://www.vinissimus.com/es/vino/les-cousins-l-inconscient/',
 'https://www.vinissimus.com/es/vino/o-luar-do-sil/',
 'https://www.vinissimu

In [4]:
# Obtain the total number of wines 

number_string = soup.find_all("span", attrs={"class":"total-count"})[0].get_text()
num = ""
for c in number_string:
    if c.isdigit():
        num = num + c
total_number = int(num)
total_number

7100

In [5]:
pages = list(range (36, total_number, 36))
pages

[36,
 72,
 108,
 144,
 180,
 216,
 252,
 288,
 324,
 360,
 396,
 432,
 468,
 504,
 540,
 576,
 612,
 648,
 684,
 720,
 756,
 792,
 828,
 864,
 900,
 936,
 972,
 1008,
 1044,
 1080,
 1116,
 1152,
 1188,
 1224,
 1260,
 1296,
 1332,
 1368,
 1404,
 1440,
 1476,
 1512,
 1548,
 1584,
 1620,
 1656,
 1692,
 1728,
 1764,
 1800,
 1836,
 1872,
 1908,
 1944,
 1980,
 2016,
 2052,
 2088,
 2124,
 2160,
 2196,
 2232,
 2268,
 2304,
 2340,
 2376,
 2412,
 2448,
 2484,
 2520,
 2556,
 2592,
 2628,
 2664,
 2700,
 2736,
 2772,
 2808,
 2844,
 2880,
 2916,
 2952,
 2988,
 3024,
 3060,
 3096,
 3132,
 3168,
 3204,
 3240,
 3276,
 3312,
 3348,
 3384,
 3420,
 3456,
 3492,
 3528,
 3564,
 3600,
 3636,
 3672,
 3708,
 3744,
 3780,
 3816,
 3852,
 3888,
 3924,
 3960,
 3996,
 4032,
 4068,
 4104,
 4140,
 4176,
 4212,
 4248,
 4284,
 4320,
 4356,
 4392,
 4428,
 4464,
 4500,
 4536,
 4572,
 4608,
 4644,
 4680,
 4716,
 4752,
 4788,
 4824,
 4860,
 4896,
 4932,
 4968,
 5004,
 5040,
 5076,
 5112,
 5148,
 5184,
 5220,
 5256,
 5292,


In [8]:
links_rest_pages = []
wrong_pages = []

try:
    for page in pages:
        request = requests.get(f"https://www.vinissimus.com/es/region/espana/?sort_by=%5B%7B%22field%22%3A%22opciones.in_stock%22%2C%22order%22%3A%22asc%22%7D%2C%7B%22field%22%3A%22ranking%22%2C%22order%22%3A%22asc%22%7D%5D&cursor={page}")
        soup = BeautifulSoup(request.content, 'html.parser')
        for a in soup.find_all("div", attrs={"class": "details"}):
            links_rest_pages.append(a.find("a").get("href"))
        wait_time = np.random.randint(3,7)
        sleep(wait_time)
except Exception as e:
        wrong_pages.append(page)
        print(page)
        print(e)  

    
links_rest_pages

['/es/vino/rapolao/',
 '/es/vino/jose-pariente-sauvignon-blanc/',
 '/es/vino/cutio-garnacha/',
 '/es/vino/malpaso/',
 '/es/vino/vina-zorzal-garnacha-blanca/',
 '/es/vino/bosque-de-matasnos-crianza/',
 '/es/cava/huguet-gran-reserva-brut-nature/',
 '/es/vino/honoro-vera-garnacha/',
 '/es/vino/emilio-moro/',
 '/es/vino/petalos-del-bierzo/',
 '/es/vino/arzuaga-crianza/',
 '/es/vino/coto-de-imaz-reserva/',
 '/es/vino/chivite-las-fincas-rosado/',
 '/es/vino/ramon-bilbao-crianza/',
 '/es/vino/campillo-gran-reserva/',
 '/es/vino/juan-gil-etiqueta-amarilla/',
 '/es/vino/dehesa-la-granja/',
 '/es/vino/alaya-tierra/',
 '/es/cava/gramona-imperial-gran-reserva/',
 '/es/vino/belondrade-quinta-apolonia/',
 '/es/cava/familia-oliveda-brut-nature-reserva/',
 '/es/vino/martelo-reserva/',
 '/es/vino/lalama/',
 '/es/vino/k-naia/',
 '/es/vino/lindes-de-remelluri-vinedos-de-san-vicente/',
 '/es/vino/el-perro-verde/',
 '/es/vino/vina-barredero/',
 '/es/vino/vinas-del-vero-gewurztraminer/',
 '/es/cava/rigol-br

In [9]:
len(links_rest_pages)

7064

In [10]:
# Add the beginning of the link to all links

missing_part = "https://www.vinissimus.com"

links_rest_pages_compl = [missing_part + l for l in links_rest_pages]
links_rest_pages_compl

['https://www.vinissimus.com/es/vino/rapolao/',
 'https://www.vinissimus.com/es/vino/jose-pariente-sauvignon-blanc/',
 'https://www.vinissimus.com/es/vino/cutio-garnacha/',
 'https://www.vinissimus.com/es/vino/malpaso/',
 'https://www.vinissimus.com/es/vino/vina-zorzal-garnacha-blanca/',
 'https://www.vinissimus.com/es/vino/bosque-de-matasnos-crianza/',
 'https://www.vinissimus.com/es/cava/huguet-gran-reserva-brut-nature/',
 'https://www.vinissimus.com/es/vino/honoro-vera-garnacha/',
 'https://www.vinissimus.com/es/vino/emilio-moro/',
 'https://www.vinissimus.com/es/vino/petalos-del-bierzo/',
 'https://www.vinissimus.com/es/vino/arzuaga-crianza/',
 'https://www.vinissimus.com/es/vino/coto-de-imaz-reserva/',
 'https://www.vinissimus.com/es/vino/chivite-las-fincas-rosado/',
 'https://www.vinissimus.com/es/vino/ramon-bilbao-crianza/',
 'https://www.vinissimus.com/es/vino/campillo-gran-reserva/',
 'https://www.vinissimus.com/es/vino/juan-gil-etiqueta-amarilla/',
 'https://www.vinissimus.co

In [11]:
# Join the two lists  

links_spanish = links_page_one_compl + links_rest_pages_compl
links_spanish

['https://www.vinissimus.com/es/vino/pago-de-carraovejas/',
 'https://www.vinissimus.com/es/vino/jose-pariente-verdejo/',
 'https://www.vinissimus.com/es/vino/ultreia-saint-jacques/',
 'https://www.vinissimus.com/es/vino/psi/',
 'https://www.vinissimus.com/es/vino/la-montesa/',
 'https://www.vinissimus.com/es/vino/louro/',
 'https://www.vinissimus.com/es/vino/vina-tondonia-reserva/',
 'https://www.vinissimus.com/es/vino/vina-ardanza-reserva/',
 'https://www.vinissimus.com/es/vino/marques-de-murrieta-reserva/',
 'https://www.vinissimus.com/es/vino/domaines-lupier-el-terroir/',
 'https://www.vinissimus.com/es/vino/emilio-moro-finca-resalso/',
 'https://www.vinissimus.com/es/vino/pago-de-los-capellanes-roble/',
 'https://www.vinissimus.com/es/vino/nekeas-cepa-por-cepa-garnacha/',
 'https://www.vinissimus.com/es/vino/vina-tondonia-blanco-reserva/',
 'https://www.vinissimus.com/es/vino/les-cousins-l-inconscient/',
 'https://www.vinissimus.com/es/vino/o-luar-do-sil/',
 'https://www.vinissimu

In [12]:
df = pd.DataFrame(links_spanish)
df

,0
0,https://www.vinissimus.com/es/vino/pago-de-car...
1,https://www.vinissimus.com/es/vino/jose-parien...
2,https://www.vinissimus.com/es/vino/ultreia-sai...
3,https://www.vinissimus.com/es/vino/psi/
4,https://www.vinissimus.com/es/vino/la-montesa/
...,...
7095,https://www.vinissimus.com/es/vino/cueva-del-m...
7096,https://www.vinissimus.com/es/vino/don-px-toro...
7097,https://www.vinissimus.com/es/vino/carlania-el...
7098,https://www.vinissimus.com/es/vino/vina-tondon...


In [13]:
df.to_csv("links_spanish.csv")

In [14]:
# Function to get name of wine and price

def obtain_wine_price(soup):
    
    # Define lists for each feature I want to obtain
    name = []
    price = []

    # Assing elements to each list
    name.append(soup.find_all("div", attrs={"class": "product-title"})[0].get_text()) 
    price.append(get_price(soup.find_all("p", attrs={"class": "price uniq large"}), 0)) 

    # Create dictionary that stores the elements
    dct = {
        "name": name,
        "price": price,
    }

    return dct

In [17]:
# Obtain the info for all wines

price_df = pd.DataFrame()
wrong_links_price = []

for link in links_spanish:
    try:
        request = requests.get(link)
        soup = BeautifulSoup(request.content, 'html.parser')
        price_info_dct = obtain_wine_price(soup)
        wait_time = np.random.randint(3,7)
        sleep(wait_time)
    except Exception as e:
        wrong_links_price.append(link)
        print(link)
        print(e)
    else:
        new_df = pd.DataFrame.from_dict(price_info_dct)
        new_df["url"] = link
        price_df = pd.concat([price_df, new_df])    

price_df.to_csv("price_euros.csv", index=False)

https://www.vinissimus.com/es/vino/formiga-de-seda-blanc/
list index out of range


In [18]:
price_df

,name,price,url
0,Pago de Carraovejas 2020,"37,40 €",https://www.vinissimus.com/es/vino/pago-de-car...
0,José Pariente Verdejo 2022,"10,50 €",https://www.vinissimus.com/es/vino/jose-parien...
0,Ultreia Saint Jacques 2021,"11,50 €",https://www.vinissimus.com/es/vino/ultreia-sai...
0,PSI 2020,"37,95 €",https://www.vinissimus.com/es/vino/psi/
0,La Montesa 2019,"13,60 €",https://www.vinissimus.com/es/vino/la-montesa/
...,...,...,...
0,Cueva del Monge Blanco 2016,"16,90 €",https://www.vinissimus.com/es/vino/cueva-del-m...
0,Don PX Toro de Albalá Reserva Selección 1968,"173,20 €",https://www.vinissimus.com/es/vino/don-px-toro...
0,Carlania Els Plans 2014,"15,95 €",https://www.vinissimus.com/es/vino/carlania-el...
0,Viña Tondonia Gran Reserva Rosado 2012,"45,00 €",https://www.vinissimus.com/es/vino/vina-tondon...


In [19]:
new_row = {"name":"Formiga de Seda Blanc 2021", "price":"15,90 €", "url":"https://www.vinissimus.com/es/vino/formiga-de-seda-blanc/"}
price_df = price_df.append(new_row, ignore_index=True)
price_df.tail()

C:\Users\mboll\AppData\Local\Temp\ipykernel_13064\1700749636.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  price_df = price_df.append(new_row, ignore_index=True)


,name,price,url
7095,Don PX Toro de Albalá Reserva Selección 1968,"173,20 €",https://www.vinissimus.com/es/vino/don-px-toro...
7096,Carlania Els Plans 2014,"15,95 €",https://www.vinissimus.com/es/vino/carlania-el...
7097,Viña Tondonia Gran Reserva Rosado 2012,"45,00 €",https://www.vinissimus.com/es/vino/vina-tondon...
7098,Adaras Lluvia 2022,"5,40 €",https://www.vinissimus.com/es/vino/adaras-cali...
7099,Formiga de Seda Blanc 2021,"15,90 €",https://www.vinissimus.com/es/vino/formiga-de-...


In [20]:
price_df.to_csv("price_euros.csv", index=False)